# Libraries

In [12]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
from math import sqrt

# reading data
import os
import json
from collections import defaultdict

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.fft import rfft, irfft, fftn, ifftn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# eFormer
from eFormer.embeddings import Encoding, ProbEncoding, PositionalEncoding
from eFormer.sparse_attention import ProbSparseAttentionModule, DetSparseAttentionModule
from eFormer.loss_function import crps
from eFormer.sparse_decoder import DetSparseDecoder, ProbSparseDecoder
from eFormer.Dataloader import TimeSeriesDataProcessor

%store -r Kelmarsh_df Penmanshiel_df

# Architektur

## Hyperparameters

In [2]:
# set global parameters

n_heads_global = 4
probabilistic_model = True
len_embedding_vector = 64

## Embedding

probabilistic embedding & positional encoding

In [16]:
test_df = Kelmarsh_df['1'][-1024:]
""" 
# First, ensure that the column is in datetime format
test_df['# Date and time'] = pd.to_datetime(test_df['# Date and time'])

# Then convert it to timestamps
test_df['Timestamp'] = test_df['# Date and time'].apply(lambda x: x.timestamp())

# interpolate NaN values
test_df = test_df.interpolate(method='linear')
""" 
features_matrix = test_df.values

%store test_df

Stored 'test_df' (DataFrame)


In [17]:
# Forward pass through the model
feature_tensor = torch.tensor(features_matrix, dtype=torch.float32).unsqueeze(0)
# check for NaN values early
if torch.isnan(feature_tensor).any():
    raise ValueError('NaN values detected in Input')

# decide which model to use
if probabilistic_model == True:
    encoding_model = ProbEncoding(in_features=feature_tensor.shape[-1], out_features=len_embedding_vector)
else:
    encoding_model = Encoding(in_features=feature_tensor.shape[-1], out_features=len_embedding_vector)

# create embeddings
embeddings = encoding_model(feature_tensor)

# Check for NaN values after computation
if torch.isnan(embeddings).any():
    raise ValueError('NaN values detected in Embeddings')
else:
    print(f"Embedding shape: {embeddings.shape}")

%store embeddings

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

## Attention Mechanism

In [5]:
# determine which model to use
if probabilistic_model == True:
    model = ProbSparseAttentionModule(
        d_model=embeddings.shape[-1],
        n_heads=n_heads_global,
        prob_sparse_factor=5
        )
else:
    model = DetSparseAttentionModule(
        d_model=embeddings.shape[-1],
        n_heads=n_heads_global,
        prob_sparse_factor=5
        )

output = model(embeddings, embeddings, embeddings)

# check for NaN values early
if torch.isnan(output).any():
    raise ValueError('NaN values detected in ProbSparse Output')
else:
    print(f"Sparse Attention shape: {output.shape}")

%store output

Sparse Attention shape: torch.Size([2, 1, 1024, 64])
Stored 'output' (Tensor)


In [6]:
# determine which model to use
if probabilistic_model == True:
    model = ProbSparseDecoder(
        d_model = len_embedding_vector,
        n_heads = n_heads_global,
        forecast_horizon = 1,
        encoder_output_dim = output[0].shape
    )
else:
    model = DetSparseDecoder(
        d_model = len_embedding_vector,
        n_heads = n_heads_global,
        forecast_horizon = 1,
        encoder_output_dim = output[0].shape
    )

forecasts = model(output)

%store forecasts

Stored 'forecasts' (Tensor)


# Transformer Model

In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe):
        # Assuming the first column is the target variable
        self.labels = dataframe.iloc[:, 0].values
        self.features = dataframe.iloc[:, 1:].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Convert data to PyTorch tensors
        features = torch.tensor(self.features[idx], dtype=torch.float)
        labels = torch.tensor(self.labels[idx], dtype=torch.float)
        return features, labels

# Assuming df_train, df_test, df_eval are your datasets
train_dataset = TimeSeriesDataset(df_train)
test_dataset = TimeSeriesDataset(df_test)
eval_dataset = TimeSeriesDataset(df_eval)

# Create DataLoaders
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) # Shuffle the data for training, typically not needed for test/eval
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)


NameError: name 'df_train' is not defined

In [18]:
# Assuming `df` is your initial DataFrame
processor = TimeSeriesDataProcessor(
    dataframe=test_df,
    forecast=1,
    look_back=72,
    batch_size=64)
    
train_loader, test_loader, eval_loader = processor.create_dataloaders()

In [19]:
class eFormer(nn.Module):
    def __init__(self, in_features, len_embedding_vector, n_heads_global, probabilistic_model=False):
        super(eFormer, self).__init__()
        self.probabilistic_model = probabilistic_model
        self.n_heads_global = n_heads_global
        self.len_embedding_vector = len_embedding_vector

        # Initialize encoding model
        if probabilistic_model:
            self.encoding_model = ProbEncoding(in_features=in_features, out_features=len_embedding_vector)
        else:
            self.encoding_model = Encoding(in_features=in_features, out_features=len_embedding_vector)

        # Initialize attention module
        if probabilistic_model:
            self.attention_module = ProbSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads_global, prob_sparse_factor=5)
        else:
            self.attention_module = DetSparseAttentionModule(d_model=len_embedding_vector, n_heads=n_heads_global, prob_sparse_factor=5)

        # Initialize decoder
        # Assuming the decoder initialization does not actually require the output shape directly but parameters that depend on the model configuration
        if probabilistic_model:
            self.decoder = ProbSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads_global, forecast_horizon=1, encoder_output_dim=len_embedding_vector)
        else:
            self.decoder = DetSparseDecoder(d_model=len_embedding_vector, n_heads=n_heads_global, forecast_horizon=1, encoder_output_dim=len_embedding_vector)

    def forward(self, features_matrix):
        feature_tensor = torch.tensor(features_matrix, dtype=torch.float32).unsqueeze(0)
        if torch.isnan(feature_tensor).any():
            raise ValueError('NaN values detected in Input')

        embeddings = self.encoding_model(feature_tensor)
        if torch.isnan(embeddings).any():
            raise ValueError('NaN values detected in Embeddings')

        encoder_output = self.attention_module(embeddings, embeddings, embeddings)
        if torch.isnan(encoder_output).any():
            raise ValueError('NaN values detected in Sparse Attention Output')

        forecasts = self.decoder(encoder_output)
        return forecasts

In [10]:
# Example usage:
model = eFormer(
    in_features=feature_tensor.shape[-1],
    len_embedding_vector=64,
    n_heads_global=4,
    probabilistic_model=True)

forecasts = model(features_matrix)

In [23]:
model = eFormer(
    in_features=feature_tensor.shape[-1],
    len_embedding_vector=64,
    n_heads_global=4,
    probabilistic_model=True)
optimizer = AdamW(
    params = model.parameters(),
    lr=6e-4,
    weight_decay=1e-1
    )
loss_fn = crps

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad()
        predictions = model(features)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} / {num_epochs} with Loss: {loss}")

    # Evaluate your model's performance on the validation set
    model.eval()
    with torch.no_grad():
        for features, labels in test_loader:
            predictions = model(features)
            # Calculate and print validation metrics


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

# Test Area